In [1]:
# Autocompletion
%config Completer.use_jedi = False

# Autoreload
%load_ext autoreload
%autoreload 2

In [2]:
"""
Notebook checks the best model and whether the generations are reproducible
"""

import pandas as pd

In [3]:
df = pd.read_csv("model-benchmark.csv")

In [4]:
df.head()

,inputs,outputs,batch_latency,batch_prompt_tokens,batch_completion_tokens,batch_tps,avg_tps,avg_latency,avg_prompt_tokens,avg_completion_tokens,...,tokenizer_decode_args,pt,pt_cols,system_prompt,batch_size,seed,bm_sample_size,gen_kwargs,exp_name,model_loading_time
0,['<|im_start|>system\nSolve the following math...,['<|im_start|> system\nSolve the following mat...,"[1.7479581832885742, 4.561641693115234, 4.0065...","[245, 242, 290, 256, 276, 298, 245, 294, 243, ...","[97, 323, 281, 89, 129, 226, 105, 166, 100, 15...","[55.49331839135082, 70.80784106465342, 70.1342...",66.551802,2.011028,251.200000,134.630000,...,{},Q: There are 15 trees in the grove. Grove work...,['question'],"Solve the following math problems, end with Th...",1,42,100,"{'max_new_tokens': 500, 'stopping_criteria': [...",exllama with batch size - 1,1.590802
1,['<|im_start|>system\nSolve the following math...,['<|im_start|> system\nSolve the following mat...,"[5.71248722076416, 4.747715950012207, 4.023427...","[245, 290, 298, 294, 272, 247, 281, 262, 239, ...","[319, 278, 226, 166, 156, 182, 210, 152, 112, ...","[55.84257568935573, 58.554471861208384, 56.171...",56.566994,3.014931,261.580000,171.000000,...,{},Q: There are 15 trees in the grove. Grove work...,['question'],"Solve the following math problems, end with Th...",2,42,100,"{'max_new_tokens': 500, 'stopping_criteria': [...",exllama with batch size - 2,1.642505
2,['<|im_start|>system\nSolve the following math...,"[""<|im_start|> system\nSolve the following mat...","[6.314980745315552, 4.996972560882568, 4.08745...","[290, 298, 272, 281, 296, 260, 258, 269, 275, ...","[278, 226, 185, 210, 285, 157, 103, 223, 202, ...","[44.02230366358919, 45.22738463068201, 45.2604...",43.916301,4.496622,271.440000,197.360000,...,{},Q: There are 15 trees in the grove. Grove work...,['question'],"Solve the following math problems, end with Th...",4,42,100,"{'max_new_tokens': 500, 'stopping_criteria': [...",exllama with batch size - 4,1.328909
3,['<|im_start|>system\nSolve the following math...,"[""<|im_start|> system\nSolve the following mat...","[12.229097127914429, 7.081205368041992, 11.567...","[298, 281, 296, 269, 275, 284, 348, 271, 256, ...","[345, 210, 330, 295, 202, 248, 267, 233, 167, ...","[28.21140403018754, 29.65596802879714, 28.5290...",29.769299,8.812006,282.307692,255.153846,...,{},Q: There are 15 trees in the grove. Grove work...,['question'],"Solve the following math problems, end with Th...",8,42,100,"{'max_new_tokens': 500, 'stopping_criteria': [...",exllama with batch size - 8,1.888971
4,['<|im_start|>system\nSolve the following math...,['<|im_start|> system\nSolve the following mat...,"[27.615971326828003, 37.8080472946167, 2.36040...","[298, 296, 284, 348, 285, 276, 258]","[367, 500, 14, 316, 279, 248, 183]","[13.289411248898265, 13.224697803189443, 5.931...",16.761614,19.258109,292.142857,272.428571,...,{},Q: There are 15 trees in the grove. Grove work...,['question'],"Solve the following math problems, end with Th...",16,42,100,"{'max_new_tokens': 500, 'stopping_criteria': [...",exllama with batch size - 16,1.626216


In [5]:
len(df)

24

In [6]:
df.columns

Index(['inputs', 'outputs', 'batch_latency', 'batch_prompt_tokens',
       'batch_completion_tokens', 'batch_tps', 'avg_tps', 'avg_latency',
       'avg_prompt_tokens', 'avg_completion_tokens', 'total_time',
       'exllama_v2', 'torch', 'transformers', 'model_id',
       'model_loader_kwargs', 'tokenizer_loader_kwargs', 'model_backend',
       'model_branch', 'tokenizer_encode_args',
       'tokenizer_decode_args', 'pt', 'pt_cols', 'system_prompt',
       'batch_size', 'seed', 'bm_sample_size', 'gen_kwargs', 'exp_name',
       'model_loading_time'],
      dtype='object')

In [7]:
df.bm_sample_size.value_counts()

100    22
5       2
Name: bm_sample_size, dtype: int64

In [8]:
df[df.bm_sample_size == 100].sort_values("total_time", ascending=True).head()[['avg_latency', 'avg_tps', 'model_backend', 'avg_prompt_tokens', 'avg_completion_tokens', 'total_time', 'batch_size', 'exp_name']]

,avg_latency,avg_tps,model_backend,avg_prompt_tokens,avg_completion_tokens,total_time,batch_size,exp_name
23,9.684867,33.316969,awq,292.142857,314.857143,67.794072,16,model with autoawq backend - 6
22,5.965763,41.009671,awq,282.307692,247.461538,77.554922,8,model with autoawq backend - 5
2,4.496622,43.916301,exllama_2_hf,271.440000,197.360000,112.415548,4,exllama with batch size - 4
3,8.812006,29.769299,exllama_2_hf,282.307692,255.153846,114.556080,8,exllama with batch size - 8
21,4.675549,47.964838,awq,271.440000,227.880000,116.888733,4,model with autoawq backend - 4


In [9]:
df[df.bm_sample_size == 100].sort_values("avg_latency", ascending=True).head()[['avg_latency', 'avg_tps', 'model_backend', 'avg_prompt_tokens', 'avg_completion_tokens', 'total_time', 'batch_size', 'exp_name']]

,avg_latency,avg_tps,model_backend,avg_prompt_tokens,avg_completion_tokens,total_time,batch_size,exp_name
0,2.011028,66.551802,exllama_2_hf,251.20,134.63,201.102797,1,exllama with batch size - 1
19,2.619404,53.607919,awq,251.20,143.36,261.940404,1,model with autoawq backend - 2
1,3.014931,56.566994,exllama_2_hf,261.58,171.00,150.746536,2,exllama with batch size - 2
11,3.494411,38.775696,auto_gptq,251.20,134.79,349.441073,1,gptq model with autogptq backend - 2
6,3.525696,38.412029,hf,251.20,135.85,352.569644,1,gptq model with hf backend with batch size - 1


In [10]:
import re
from ast import literal_eval

In [11]:
df_awq = df[(df.bm_sample_size == 100) & (df.model_backend == 'awq')]
len(df_awq)

5

In [12]:
df_awq[['avg_latency', 'avg_tps', 'model_backend', 'avg_prompt_tokens', 'avg_completion_tokens', 'total_time', 'batch_size', 'exp_name']]

,avg_latency,avg_tps,model_backend,avg_prompt_tokens,avg_completion_tokens,total_time,batch_size,exp_name
19,2.619404,53.607919,awq,251.200000,143.360000,261.940404,1,model with autoawq backend - 2
20,3.982445,47.131561,awq,261.580000,193.000000,199.122238,2,model with autoawq backend - 3
21,4.675549,47.964838,awq,271.440000,227.880000,116.888733,4,model with autoawq backend - 4
22,5.965763,41.009671,awq,282.307692,247.461538,77.554922,8,model with autoawq backend - 5
23,9.684867,33.316969,awq,292.142857,314.857143,67.794072,16,model with autoawq backend - 6


In [13]:
def extract_outputs(s):
    out = []
    pattern = re.compile(r"The answer is (\d+(?:\.\d+)?)")

    raw_outs = literal_eval(s)

    for item in raw_outs:
        model_out = item.split('<|im_start|> assistant')[-1]
        match = pattern.search(model_out)
        if match:
            out.append(float(match.group(1).strip()))
        else:
            out.append(None)

    return out

out = extract_outputs(df_awq.loc[19].outputs)


In [14]:
extract_outputs(df_awq.loc[19].outputs)

[16.0,
 None,
 191.0,
 None,
 15.0,
 40.0,
 75.0,
 7.0,
 3.0,
 55.0,
 2.0,
 160.0,
 None,
 2.0,
 None,
 None,
 310.0,
 12.0,
 22.0,
 5.0,
 45.0,
 None,
 143.0,
 None,
 3.0,
 50.0,
 320.0,
 960.0,
 35.0,
 1260.0,
 23.0,
 53.0,
 20.0,
 14.0,
 5.0,
 14.0,
 None,
 30.0,
 None,
 30.0,
 36.0,
 30.0,
 3500.0,
 None,
 5.0,
 140.0,
 20.0,
 None,
 52.0,
 20.0,
 8.0,
 80.0,
 230.0,
 148.0,
 6.0,
 22.0,
 50.0,
 228.0,
 490.0,
 12.0,
 36.0,
 26.0,
 14.0,
 1.0,
 20.0,
 18.0,
 55.0,
 None,
 None,
 25.0,
 75.0,
 40.0,
 34.0,
 52.0,
 11.0,
 33.0,
 25.0,
 7.0,
 11.0,
 136.0,
 77.0,
 22.0,
 None,
 40.0,
 15.0,
 45.0,
 42.0,
 14.0,
 85.0,
 75.0,
 21.0,
 10.0,
 100.0,
 16.0,
 9.0,
 10.0,
 240.0,
 360.0,
 6.0,
 None]

In [76]:
for item in literal_eval(df_awq.loc[19].outputs):
    print(item)

    print('\n\n')
    print('-----' * 10)
    print('\n\n')

<|im_start|> system
Solve the following math problems, end with The answer is<|im_end|>
 <|im_start|> user
Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
A: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. The answer is 6.

Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
A: There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. The answer is 5.

Q: Mimi picked up 2 dozen seashells on the beach.  Kyle found twice as many shells as Mimi and put them in his pocket. Leigh grabbed one-third of the shells that Kyle found.  How many seashells did Leigh have?<|im_end|>
 <|im_start|> assistant
A: Mimi picked up 2 dozen seashells, which is 2 * 12 = 24 seashells. Kyle found twice as many shells as Mimi, so he found 24 * 2 = 48 seas

In [16]:
for item in literal_eval(df_awq.loc[21].outputs):
    print(item)

    print('\n\n')
    print('-----' * 10)
    print('\n\n')

_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><s>  <|im_start|> system
Solve the following math problems, end with The answer is<|im_end|>
 <|im_start|> user
Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
A: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. The answer is 6.

Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
A: There are